In [1]:
#Import Modules
import pandas as pd

#### Load raw data

In [2]:
df_train = pd.read_csv('data/train_data.csv')
df_test = pd.read_csv('data/test_data.csv')
df_train_labels = pd.read_csv('data/train_labels.csv')

In [6]:
df_train = df_train.drop('is_duplicate', axis = 1)
df_train = df_train.merge(df_train_labels)
df_train.head()

,id,question1,question2,is_duplicate
0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


#### Checklist:
- Last Character (Question more likely to be duplicate if they end with same punctuation mark
- Average Shared Words - Words shared by question pairs (more words shared = higher chance of duplicate)
- Shared Entities (Enzo-engineered entities using spaCy)
- Length (word count) - Words per sentence (Questions with different lengths they are unlikely to be duplicates) 
- Levenshtein Features - How many characters difference between two questions (more functions required = more dissimilar)
- Tf-Idf 
- LDA (topic modelling)


https://www.linkedin.com/pulse/kaggle-quora-question-pairs-mar-2017-may-priscilla-li/